In [1]:
import pandas as pd
import plotly.express as px

# Load the data
df = pd.read_csv('/Users/rayreyes/Library/CloudStorage/OneDrive-GettysburgCollege/Data Visualization/dashboard data/State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month (1).csv')

# Identify date columns
date_cols = [col for col in df.columns if '-' in col]

# Melt the dataframe to long format
df_long = df.melt(id_vars=['RegionName'], value_vars=date_cols, var_name='Date', value_name='AvgPrice')

# Convert Date to datetime
df_long['Date'] = pd.to_datetime(df_long['Date'])

# Map state names to state codes
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df_long['StateCode'] = df_long['RegionName'].map(state_abbrev)

# Drop rows with missing state codes
df_long = df_long.dropna(subset=['StateCode'])

# Create animated map
fig = px.choropleth(
    df_long,
    locations='StateCode',
    locationmode='USA-states',
    color='AvgPrice',
    animation_frame=df_long['Date'].dt.strftime('%Y-%m'),
    scope='usa',
    color_continuous_scale='Viridis',
    labels={'AvgPrice': 'Average House Price ($)'}
)

fig.update_layout(
    title_text='Average House Price by State (2000-2025)',
    geo=dict(showlakes=True, lakecolor='rgb(255, 255, 255)'),
    coloraxis_colorbar=dict(title="Avg Price ($)"),
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig.show()

In [2]:
import pandas as pd
import plotly.express as px

# === 1. Load your already cleaned data ===
df_cleaned = pd.read_csv('/Users/rayreyes/Library/CloudStorage/OneDrive-GettysburgCollege/Data Visualization/dashboard data/Cleaned_Housing_Price_Data.csv')  # or use the dataframe we created earlier

# If already cleaned (RegionName, Date, AvgPrice, StateCode), no need to re-melt!

# Ensure Date is datetime
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])

# === 2. No dropping based on missing AvgPrice, only drop missing StateCode ===
# (Assuming you already have StateCode, but if you need, map again.)

# === 3. Optional: Forward fill missing prices for smoother animation ===
df_cleaned['AvgPrice'] = df_cleaned.groupby('StateCode')['AvgPrice'].fillna(method='ffill')

# === 4. Set up fixed color scale ===
price_min = df_cleaned['AvgPrice'].min()
price_max = df_cleaned['AvgPrice'].max()

# === 5. Create the animated choropleth ===
fig = px.choropleth(
    df_cleaned,
    locations='StateCode',
    locationmode='USA-states',
    color='AvgPrice',
    animation_frame=df_cleaned['Date'].dt.strftime('%Y-%m'),
    scope='usa',
    color_continuous_scale='jet',
    range_color=[price_min, price_max],    # fix color range!
    labels={'AvgPrice': 'Average House Price ($)'},
    hover_name='RegionName',                # clean hover
    hover_data={'StateCode': False, 'Date': False}  # hide unnecessary hover fields
)

fig.update_layout(
    title_text='Average House Price by State (2000–2025)',
    geo=dict(
        showlakes=True,
        lakecolor='rgb(255, 255, 255)'
    ),
    coloraxis_colorbar=dict(
        title="Avg House Price ($)",
    ),
    margin={"r":0,"t":50,"l":0,"b":0},
    sliders=[{
        'currentvalue': {
            'prefix': 'Date: ',
            'font': {'size': 18}
        }
    }],
    updatemenus=[{
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 100, 'redraw': True}, 'fromcurrent': True}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }]
)

fig.show()


/var/folders/8z/q7b20vdn3z733m_0v20wnn_r0000gn/T/ipykernel_14583/1226500673.py:16: FutureWarning:

SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead

/var/folders/8z/q7b20vdn3z733m_0v20wnn_r0000gn/T/ipykernel_14583/1226500673.py:16: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [4]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Output, Input
import dash_bootstrap_components as dbc

# === 1. Load the already cleaned master dataset ===
df_master = pd.read_csv('/Users/rayreyes/Library/CloudStorage/OneDrive-GettysburgCollege/Data Visualization/dashboard data/Master_Housing_Price_Dataset_with_Bedrooms.csv')  # <- use your real file path here

# Fix Date column
df_master['Date'] = pd.to_datetime(df_master['Date'])

# App setup
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Bedroom options
bedroom_options = df_master['Bedrooms'].unique().tolist()
bedroom_options = sorted(bedroom_options, key=lambda x: (str(x) != 'All', x))  # Sort with 'All' first

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("US Housing Prices Dashboard", style={'textAlign': 'center'}),
            dcc.Tabs(id='tabs', value='All', children=[
                dcc.Tab(label=f'{b} Bedrooms' if b != 'All' else 'All Homes', value=b) for b in bedroom_options
            ]),
            dcc.Loading(
                dcc.Graph(id='choropleth-map'),
                type="circle"
            )
        ])
    ])
], fluid=True)

@app.callback(
    Output('choropleth-map', 'figure'),
    Input('tabs', 'value')
)
def update_map(selected_bedrooms):
    filtered_df = df_master[df_master['Bedrooms'] == selected_bedrooms]

    fig = px.choropleth(
        filtered_df,
        locations='StateCode',
        locationmode='USA-states',
        color='AvgPrice',
        animation_frame=filtered_df['Date'].dt.strftime('%Y-%m'),
        scope='usa',
        color_continuous_scale='jet',
        range_color=[filtered_df['AvgPrice'].min(), filtered_df['AvgPrice'].max()],
        labels={'AvgPrice': 'Avg Price ($)'},
        hover_name='RegionName'
    )

    fig.update_layout(
        title_text=f'Average House Price - {selected_bedrooms} Bedrooms',
        geo=dict(showlakes=True, lakecolor='rgb(255, 255, 255)'),
        margin={"r":0,"t":50,"l":0,"b":0}
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [6]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Output, Input, dash_table
import dash_bootstrap_components as dbc

# Load the already cleaned master dataset
df_master = pd.read_csv('/Users/rayreyes/Library/CloudStorage/OneDrive-GettysburgCollege/Data Visualization/dashboard data/Master_Housing_Price_Dataset_with_Bedrooms.csv')  # Replace with your actual master file path

# Fix Date column
df_master['Date'] = pd.to_datetime(df_master['Date'])

# App setup
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Bedroom options
bedroom_options = df_master['Bedrooms'].unique().tolist()
bedroom_options = sorted(bedroom_options, key=lambda x: (str(x) != 'All', x))  # Sort with 'All' first

# Unique Dates for slider
date_options = df_master['Date'].dt.strftime('%Y-%m').sort_values().unique()

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("US Housing Prices Dashboard", style={'textAlign': 'center'}),
            dcc.Tabs(id='tabs', value='All', children=[
                dcc.Tab(label=f'{b} Bedrooms' if b != 'All' else 'All Homes', value=b) for b in bedroom_options
            ]),
            dcc.Slider(
    id='month-slider',
    min=0,
    max=len(date_options)-1,
    value=0,
    marks={i: date[:4] for i, date in enumerate(date_options) if date.endswith('-01')},
    step=1,
    tooltip={"placement": "bottom", "always_visible": True}
)

        ])
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Loading(dcc.Graph(id='choropleth-map'), type="circle")
        ], width=8),
        dbc.Col([
            dcc.Loading(dash_table.DataTable(
                id='price-table',
                columns=[
                    {"name": "State", "id": "RegionName"},
                    {"name": "Average Price ($)", "id": "AvgPrice", "type": "numeric", "format": {"locale": {"symbol": ["$", ""]}}}
                ],
                style_table={'height': '600px', 'overflowY': 'auto'},
                style_cell={'textAlign': 'left', 'padding': '5px'},
                style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
                sort_action='native'
            ), type="circle")
        ], width=4)
    ])
], fluid=True)

@app.callback(
    [Output('choropleth-map', 'figure'), Output('price-table', 'data')],
    [Input('tabs', 'value'), Input('month-slider', 'value')]
)
def update_dashboard(selected_bedrooms, selected_month_idx):
    selected_month = pd.to_datetime(date_options[selected_month_idx])

    filtered_df = df_master[(df_master['Bedrooms'] == selected_bedrooms) & (df_master['Date'].dt.strftime('%Y-%m') == selected_month.strftime('%Y-%m'))]

    # Map
    fig = px.choropleth(
        filtered_df,
        locations='StateCode',
        locationmode='USA-states',
        color='AvgPrice',
        scope='usa',
        color_continuous_scale='jet',
        range_color=[filtered_df['AvgPrice'].min(), filtered_df['AvgPrice'].max()],
        labels={'AvgPrice': 'Avg Price ($)'},
        hover_name='RegionName'
    )

    fig.update_layout(
        title_text=f'Average House Price - {selected_bedrooms} Bedrooms ({selected_month.strftime("%Y-%m")})',
        geo=dict(showlakes=True, lakecolor='rgb(255, 255, 255)'),
        margin={"r":0,"t":50,"l":0,"b":0}
    )

    # Table
    table_data = filtered_df[['RegionName', 'AvgPrice']].sort_values(by='AvgPrice', ascending=False).to_dict('records')

    return fig, table_data

if __name__ == '__main__':
    app.run_server(debug=True)